In [1]:
import re, json, pathlib, logging, time, argparse, pprint, math 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr
import keyword


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [14]:
results_df = pd.read_excel('output/SDGS_61_Green_Deal_Test/output/6-results/mapping_.xlsx', sheet_name='Summary', index_col=0)
results_df[results_df.Goal == 'SDG 2'].head(20)

,Goal,Target,Sum_of_keys,Count_of_keys,list_of_keys
Index,,,,,
9,SDG 2,2.0,0,0,NaN
10,SDG 2,2.1,2,2,"food insecur , food all"
11,SDG 2,2.2,1,1,healthi sustain diet
12,SDG 2,2.3,1,1,decent live farmer
13,SDG 2,2.4,5,4,"organ farm , environmentally-friendli food system , agro-ecolog , sustain food system"
14,SDG 2,2.5,0,0,NaN
15,SDG 2,2.a,0,0,NaN
16,SDG 2,2.b,0,0,NaN
17,SDG 2,2.c,0,0,NaN


In [20]:
def make_bubbleplot(results_df):
    #Later on, think about a way to make bubbleplot for PP
    sdg_bubbledict={"name": "sdgs", "children": None}
    goal_ls=[]

    goals=results_df.groupby(by=["Goal"]).sum()

    for goal, count in zip(goals.index.tolist(), goals['Sum_of_keys']):
        goal_dict={}
        if count > 0:
            goal_dict["name"]=goal
            goal_dict["size"]=count
            goal_dict["children"] = None
            goal_ls.append(goal_dict)
        else:
            continue


    for goal in goal_ls:
        target_ls=[]
        if goal["size"] >0:
            targetdf = results_df[results_df.Goal == goal["name"]] 
            for target, count in zip(targetdf['Target'], targetdf['Sum_of_keys']):
                target_dict={}
                target_dict["name"]=f'Target {target}'
                target_dict["size"]=count
                target_ls.append(target_dict)
            goal['children']=target_ls
        else:
            continue

    sdg_bubbledict['children']=goal_ls

return sdg_bubbledict

with open('sdg_bubbles_sdgs61.json', mode='w', encoding='utf-8') as f:
    json.dump(sdg_bubbledict, f)

# targets=results_df[results_df.Goal=='SDG 1']
# targets.head(15)
# for target, count in zip(targets['Target'], targets['Sum_of_keys']):
#     print(target, count)

{'name': 'sdgs', 'children': [{'name': 'SDG 10', 'size': 2, 'children': [{'name': 'Target 10.0', 'size': 0}, {'name': 'Target 10.1', 'size': 0}, {'name': 'Target 10.2', 'size': 0}, {'name': 'Target 10.3', 'size': 1}, {'name': 'Target 10.4', 'size': 0}, {'name': 'Target 10.5', 'size': 0}, {'name': 'Target 10.6', 'size': 0}, {'name': 'Target 10.7', 'size': 0}, {'name': 'Target 10.a', 'size': 0}, {'name': 'Target 10.b', 'size': 1}, {'name': 'Target 10.c', 'size': 0}]}, {'name': 'SDG 11', 'size': 14, 'children': [{'name': 'Target 11.0', 'size': 0}, {'name': 'Target 11.1', 'size': 0}, {'name': 'Target 11.2', 'size': 10}, {'name': 'Target 11.3', 'size': 2}, {'name': 'Target 11.4', 'size': 0}, {'name': 'Target 11.5', 'size': 0}, {'name': 'Target 11.6', 'size': 1}, {'name': 'Target 11.7', 'size': 0}, {'name': 'Target 11.a', 'size': 1}, {'name': 'Target 11.b', 'size': 0}, {'name': 'Target 11.c', 'size': 0}]}, {'name': 'SDG 12', 'size': 40, 'children': [{'name': 'Target 12.0', 'size': 5}, {'name

In [ ]:
df = pd.read_excel('output/The_EU_Green_Deal_/output/6-results/results_.xlsx', sheet_name='priorities', index_col=0)

pp_colors={'Human Development':'#FECA4D',
'Growth Jobs':'#FABE13',
'Green Deal':'#73936F',
'Partnership':'#1A6835',
'Digital':'#72ABDC',
'Governance':'#E2809B',
'Digitalisation':'#4D9CD5',
'Migration':'#DE6189'
}

df.rename(columns = {'priority':'name', 'Count':'size'}, inplace = True)
df['goal_color']=df['name'].map(pp_colors)
df.to_dict('index')


In [ ]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

In [ ]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

In [ ]:
targets.head

In [ ]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()